# 멋진 작사가 만들기
목표: 입력값이 포함된 문장을 작사하는 모델 만들기  
## 데이터 불러오기

In [1]:
import os, re 
import numpy as np
import tensorflow as tf

import glob
import os

from sklearn.model_selection import train_test_split


txt_file_path = './lyricist/data/lyrics'    # 로컬에서는'C:/Users/wader/workplace/ess_data/[E-06]NLP_First/'
txt_list=os.listdir(txt_file_path)

os.chdir(txt_file_path)
raw_corpus = [] 

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r", encoding='UTF8') as f:
        raw = f.read().splitlines() #read() : 파일 전체의 내용을 하나의 문자열로 읽어온다. , splitlines()  : 여러라인으로 구분되어 있는 문자열을 한라인씩 분리하여 리스트로 반환
        raw_corpus.extend(raw) # extend() : 리스트함수로 추가적인 내용을 연장 한다.

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Busted flat in Baton Rouge, waitin' for a train", "And I's feelin' near as faded as my jeans", 'Bobby thumbed a diesel down, just before it rained']


## 전처리
preprocess_sentence는 대소문자, 불필요한 문장부호, 특수문자를 제거한다.

In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

불러운 문장들은 preprocess_sentence로 정제한다. 다만 단어가 15개가 넘어가는 학습에 적절하지 못한 문장으로 보아 삭제하고 남은 값들을 학습시킨다.(문장이 지나치게 긴 결과물이 나오는 것을 예방하기 위함)

In [3]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

# raw_corpus list에 저장된 문장들을 순서대로 반환하여 sentence에 저장
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 앞서 구현한 preprocess_sentence() 함수를 이용하여 문장을 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    
    # 문장에 들어가는 단어가 15개가 넘는 경우 학습 데이터에 포함하지 않는다.
    if len(preprocessed_sentence.split(' ')) >= 15:
        continue
    
    corpus.append(preprocessed_sentence)
        
# 정제된 결과 확인해보죠
corpus[:10]

['<start> busted flat in baton rouge , waitin for a train <end>',
 '<start> and i s feelin near as faded as my jeans <end>',
 '<start> bobby thumbed a diesel down , just before it rained <end>',
 '<start> i was playin soft while bobby sang the blues , yeah <end>',
 '<start> you know , feelin good was good enough for me <end>',
 '<start> there bobby shared the secrets of my soul <end>',
 '<start> through all kinds of weather , through everything we done <end>',
 '<start> nothin , that s all that bobby left me , yeah <end>',
 '<start> hey , feelin good was good enough for me , mm hmm <end>',
 '<start> good enough for me and my bobby mcghee la da da <end>']

## Test data와 Validation data 분류 및 토큰화
Test:Validation= 8:2 로 하며, 단어장은 12000단어를 저장할 수 있도록 한다.

In [4]:
enc_train, enc_val, _, _ = train_test_split(corpus, corpus, test_size=0.2, random_state=7)

In [5]:
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    # tokenizer.fit_on_texts(texts): 문자 데이터를 입력받아 리스트의 형태로 변환하는 메서드
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    # tokenizer.texts_to_sequences(texts): 텍스트 안의 단어들을 숫자의 시퀀스 형태로 변환하는 메서드
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(enc_train)

[[   2   87  545 ...    0    0    0]
 [   2   92   66 ...    0    0    0]
 [   2    4   75 ...    0    0    0]
 ...
 [   2  510 5037 ...    0    0    0]
 [   2    4  119 ...    0    0    0]
 [   2   68 1552 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f786c4cc970>


연산 결과를 확인한다. 텐서에 padding과 단어 code가 들어갔는지 확인한다.

In [6]:
print(tensor[:3, :15])

[[   2   87  545    6  193    5   51    5  868 2533    3    0    0    0]
 [   2   92   66    6  474   72    6  516    3    0    0    0    0    0]
 [   2    4   75  250  140   28  608   20  130    3    0    0    0    0]]


실제 단어장의 역할을 하는지 확인한다.

In [7]:
# tokenizer.index_word: 현재 계산된 단어의 인덱스와 인덱스에 해당하는 단어를 dictionary 형대로 반환 (Ex. {index: '~~', index: '~~', ...})
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

In [9]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

 # tokenizer가 구축한 단어사전 내 12000개와, 여기 포함되지 않은 0:<pad>를 포함하여 12001개
 # tokenizer.num_words: 주어진 데이터의 문장들에서 빈도수가 높은 n개의 단어만 선택
 # tokenize() 함수에서 num_words를 12000개로 선언했기 때문에, tokenizer.num_words의 값은 12000
VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
# 데이터셋에 대해서는 아래 문서를 참고하세요
# 자세히 알아둘수록 도움이 많이 되는 중요한 문서입니다
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)) # 첫번째 차원을 기준으로 데이터들을 slicing한다.
dataset = dataset.shuffle(BUFFER_SIZE) # 전체를 섞어주기 위해 src_input의 개수를 BUFFER_SIZE로 지정하였다.
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 13), (256, 13)), types: (tf.int32, tf.int32)>

In [10]:
src_input.shape 

(119992, 13)

모델 형성: 베이스라인을 그대로 따라감

In [11]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        # Embedding 레이어, 2개의 LSTM 레이어, 1개의 Dense 레이어로 구성되어 있다.
        # Embedding 레이어는 단어 사전의 인덱스 값을 해당 인덱스 번째의 워드 벡터로 바꿔준다.
        # 이 워드 벡터는 의미 벡터 공간에서 단어의 추상적 표현으로 사용된다. 
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size) 
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)  
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
# embedding size 값이 커질수록 단어의 추상적인 특징들을 더 잡아낼 수 있지만
# 그만큼 충분한 데이터가 없으면 안좋은 결과 값을 가져옵니다!   
embedding_size = 256 # 워드 벡터의 차원수를 말하며 단어가 추상적으로 표현되는 크기입니다.
hidden_size = 1024 # 모델에 얼마나 많은 일꾼을 둘 것인가? 정도로 이해하면 좋다.
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size) # tokenizer.num_words에 +1인 이유는 문장에 없는 pad가 사용되었기 때문이다.

In [12]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법입니다.
# 지금은 동작 원리에 너무 빠져들지 마세요~
for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)

<tf.Tensor: shape=(256, 13, 12001), dtype=float32, numpy=
array([[[ 6.55656331e-05,  1.60237279e-04, -3.92230068e-05, ...,
         -2.39942005e-04, -3.69633672e-05,  4.49047502e-06],
        [-4.67502177e-05,  1.75597597e-04,  1.53114142e-05, ...,
         -1.43033976e-04, -5.35820982e-05,  3.06380272e-04],
        [-3.46817797e-05, -2.03083226e-04,  1.25399078e-04, ...,
         -1.67871505e-04,  3.22222841e-05,  6.74389827e-04],
        ...,
        [ 1.87398796e-03, -1.02797442e-03,  5.82572015e-04, ...,
          1.28762226e-03, -2.09157399e-04, -9.32944531e-05],
        [ 2.05908227e-03, -1.16908259e-03,  3.19911662e-04, ...,
          1.23905041e-03, -5.22169168e-04, -3.86267318e-04],
        [ 2.22715922e-03, -1.28042360e-03,  5.30764155e-05, ...,
          1.19907677e-03, -8.46231356e-04, -6.93592185e-04]],

       [[ 6.55656331e-05,  1.60237279e-04, -3.92230068e-05, ...,
         -2.39942005e-04, -3.69633672e-05,  4.49047502e-06],
        [ 1.12417820e-06,  3.41815379e-04, -2

In [13]:
# 모델의 구조를 확인합니다.
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


## 학습

In [14]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다

# Adam 알고리즘을 구현하는 optimzier이며 어떤 optimzier를 써야할지 모른다면 Adam을 쓰는 것도 방법이다.
# 우리가 학습을 할 때 최대한 틀리지 않는 방향으로 학습을 해야한다.
# 여기서 얼마나 틀리는지(loss)를 알게하는 함수가 손실함수 이다.
# 이 손실함수의 최소값을 찾는 것을 학습의 목표로 하며 여기서 최소값을 찾아가는 과정을 optimization 이라하고
# 이를 수행하는 알고리즘을 optimizer(최적화)라고 한다.

optimizer = tf.keras.optimizers.Adam() # Adam은 현재 가장 많이 사용하는 옵티마이저이다. 자세한 내용은 차차 배운다.
loss = tf.keras.losses.SparseCategoricalCrossentropy( # 훈련 데이터의 라벨이 정수의 형태로 제공될 때 사용하는 손실함수이다.
    from_logits=True, # 기본값은 False이다. 모델에 의해 생성된 출력 값이 정규화되지 않았음을 손실 함수에 알려준다. 즉 softmax함수가 적용되지 않았다는걸 의미한다. 
    reduction='none'  # 기본값은 SUM이다. 각자 나오는 값의 반환 원할 때 None을 사용한다.
)
# 모델을 학습시키키 위한 학습과정을 설정하는 단계이다.
model.compile(loss=loss, optimizer=optimizer) # 손실함수와 훈련과정을 설정했다.
model.fit(dataset, epochs=30) # 만들어둔 데이터셋으로 모델을 학습한다. 30번 학습을 반복하겠다는 의미다.

Epoch 1/30
468/468 [==============================] - 74s 146ms/step - loss: 3.6592
Epoch 2/30
468/468 [==============================] - 78s 166ms/step - loss: 3.1722
Epoch 3/30
468/468 [==============================] - 79s 169ms/step - loss: 3.0001
Epoch 4/30
468/468 [==============================] - 79s 169ms/step - loss: 2.8686
Epoch 5/30
468/468 [==============================] - 79s 169ms/step - loss: 2.7559
Epoch 6/30
468/468 [==============================] - 79s 169ms/step - loss: 2.6537
Epoch 7/30
468/468 [==============================] - 79s 169ms/step - loss: 2.5588
Epoch 8/30
468/468 [==============================] - 79s 169ms/step - loss: 2.4689
Epoch 9/30
468/468 [==============================] - 79s 169ms/step - loss: 2.3842
Epoch 10/30
468/468 [==============================] - 79s 169ms/step - loss: 2.3025
Epoch 11/30
468/468 [==============================] - 79s 169ms/step - loss: 2.2258
Epoch 12/30
468/468 [==============================] - 79s 169ms/step - lo

In [15]:
import os
os.getcwd()

'/aiffel/data/lyrics'

In [16]:
#문장생성 함수 정의
#모델에게 시작 문장을 전달하면 모델이 시작 문장을 바탕으로 작문을 진행
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20): #시작 문자열을 init_sentence 로 받으며 디폴트값은 <start> 를 받는다
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence]) #텍스트 안의 단어들을 숫자의 시퀀스의 형태로 변환
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다 (도달 하지 못하였으면 while 루프를 돌면서 다음 단어를 예측)
    while True: #루프를 돌면서 init_sentence에 단어를 하나씩 생성성
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated #최종적으로 모델이 생성한 문장을 반환

In [26]:
generate_text(model, tokenizer, init_sentence="<start> he is") # 시작문장으로 he를 넣어 문장생성 함수 실행

'<start> he is he is intro the notorious big <end> '

회고
지금까지 NLP를 통해 작사 모델을 만들어 보았다.
중요 부분은 노드의 코드들을 대부분 참고했고, 텐서플로우에 대해 아는
바도 적었기 때문에 코드를 100%이해하였다고 하기에는 문제가 있다.
다만, 과제를 하며 느낀 점들을 적어본다.

1. 정규식  
예전 지인한테 들었던 말이 기었난다.
'정규식은 안 쓰는데 없고, 정규식 잘 쓰는 프로그래머는 어디를 가든 우대받는다.'
분명 복잡하고 어려워 보이지만 정규식을 잘 사용한다는 것은
조작하고자 하는 데이터의 패턴을 명확히 파악하고 설계한다는 의미일 것이다.
과제에서도 전처리 프로세스와 glob 함수를 사용하는 과정을 통해 그 중요성을
다시 한번 느낄 수 있었다. 앞으로 파이썬 기초를 다시 훑고 갈 예정인데,
정규식에 대해 좀 더 공부해 보는게 좋을 것 같다.  
  
2. NLP 학습의 대략적인 과정의 이해  
학습의 전체적인 흐름에 대해 다시 이해할 수 있는 시간이었다. NLP학습의 대략적인 흐름은 다음과 같다. 데이터 수집 및 로딩-> 데이터 정제(대소문자, 특수문자 등 처리)->토큰화 및 단어 사전 만들기->학습 데이터 정리(tf 모델에 사용 가능하도록
데이터 tensor로 변환 및 하이퍼파라미터 정리)->모델 설계->학습->테스트  
  
3. NLP 처리 및 테스트의 어려움  
어려운 주제가 선정된 것인지도 모르지만, NLP는 CV에 비해 처리 과정이나
모델 학습, 테스트 등이 훨씬 더 어려운 것 같다.
CV 같은 경우 데이터 전처리 과정에서 시각화가 쉽게 이루어지지만
NLP는 토큰화와 임배딩 과정을 거치기 때문인지 처음 하는 사람 입장에서
중간중간 데이터를 꺼내 보는 것조차 부담스러운 부분이 있었고,
학습을 시키는데도 훨씬 많은 시간이 걸렸다.
테스트의 경우에도 CV는 다양한 방식으로 확인할 수 있었지만
NLP의 경우에는 직접 단어를 입력하여 몇몇 값을 확인하는 것이 최선이었다.
(여러 단어를 한꺼번에 입력한다고 하여도 문장 하나하나를 봐야 하기 때문에
여러 단어 입력에 큰 의미도 없다.) 물론 BLEU나 ROUGE등의 방법이 존재
한다고 하지만 이미지와 달리 언어는 단순히 단어의 내용이나 배열 순서만으로
정답과 오답을 가리기 어렵기 때문에 상당히 경직된 평가 방법이라는 생각이 들었다.
그렇기 때문에 NLP가 CV보다 훨씬 어렵게 느껴졌다.  
  
4. 내려놓기  
과제를 반복할수록 내려놓아야겠다는 생각이 든다.
과정 하나하나, 코드 하나하나를 이해하려는 생각 말이다.
교육 전체를 봤을 때 상당히 실전적이라는 장점이 있으나
겪은 실전을 내 것으로 소화시키기가 상당히 어렵다.
아무래도 실적을 뒷받침할 이론이 정립되지 않았기 때문이라고 생각하는데,
이러한 부분을 아이펠에서는 반복 실전으로 해결하려는 경향이 보인다.
반복하여 전에 봤던 내용들을 조금씩 다시 제시하는 방식인 것 같은데
그렇다면 지금 디테일하게 과제나 노드들을 볼 필요는 없다는 결론이 나왔다.
어차피 나중에 또 볼테니까.
지금부터는 과제에 독창성을 부여하기 보다는 주어진 루트를 따라가고
전체적인 논리를 이해하는 방향으로 초점을 맞출까 한다.
  
5. 중요 키워드  
정규식, 토큰화, 임배딩(레이어), LSTM